## Evaluación de Modelos de Representación Semántica para Textos de Recalls Automotrices

### 1. Contexto y objetivo

El presente análisis tuvo como propósito identificar el modelo de *text embedding* más adecuado para representar y recuperar información técnica a partir de **textos de recalls** emitidos por la *National Highway Traffic Safety Administration (NHTSA)*.  
Estos textos constituyen descripciones detalladas de fallas, causas, consecuencias y medidas correctivas en campañas de seguridad vehicular, y representan un lenguaje técnico especializado que combina terminología de ingeniería y redacción administrativa.

El objetivo general fue determinar cuál modelo de representación semántica permite **maximizar la coherencia entre textos de naturaleza técnica y lingüísticamente heterogénea**, preparando el terreno para la integración posterior de **complaints** (quejas de usuarios) y **manuales técnicos** en un espacio vectorial común.

---

### 2. Preparación y procesamiento de los datos

A partir del archivo `recalls_corpus.jsonl`, que contenía aproximadamente **12,871 pasajes** (chunks), se extrajeron los campos `id`, `text` y `metadata`, generando una colección representativa de textos con extensión promedio de 200–300 tokens.  
El proceso de *chunking* se implementó para mantener coherencia semántica y reducir la dilución contextual, permitiendo al modelo capturar información localizada sobre componentes, consecuencias y remedios técnicos.

Los *chunks* resultantes se almacenaron en formato Parquet (`recalls_chunks_fixed.parquet`) y constituyeron la base del experimento comparativo.

---

### 3. Metodología de evaluación

Se evaluaron diversos modelos de *embedding* mediante un esquema **retrieval-to-retrieval (R2R)**, empleando subconjuntos aleatorios de *queries* extraídas del mismo dominio.  
Cada modelo generó representaciones vectoriales que se indexaron con FAISS (HNSW), y se midió la calidad del *retrieval* mediante tres métricas estándar en el campo:

- **Recall@10**: proporción de consultas cuyo documento relevante aparece entre los 10 más cercanos en el espacio vectorial.  
- **Mean Reciprocal Rank (MRR@10)**: media del inverso del rango del primer documento relevante, sensible al orden del resultado.  
- **Normalized Discounted Cumulative Gain (nDCG@10)**: métrica ponderada que captura la ganancia acumulada según la posición de los resultados, penalizando documentos relevantes a menor rango.

Estas métricas se emplean ampliamente en benchmarks como **MTEB** (*Massive Text Embedding Benchmark*) y constituyen un indicador confiable de la capacidad de un modelo para recuperar información semánticamente coherente.

---

### 4. Modelos evaluados

Los modelos seleccionados cubren un espectro representativo de arquitecturas y tamaños:

| Modelo | Parámetros | Dimensiones | Orientación |
|---------|-------------|--------------|--------------|
| BAAI/bge-m3 | 568M | 1024 | Multitarea, inglés/multilingüe |
| intfloat/multilingual-e5-large-instruct | 560M | 1024 | Multilingüe instruccional |
| intfloat/e5-base-v2 | 300M | 768 | Inglés general |
| sentence-transformers/all-MiniLM-L6-v2 | 66M | 384 | Ligero, inglés general |
| mixedbread-ai/mxbai-embed-large-v1 | ≈1B | 1024 | Inglés técnico y general |
| nomic-ai/ModernBERT-base | 125M | 768 | BERT moderno optimizado |

Cada modelo se ejecutó sobre GPU (NVIDIA T4) con los mismos parámetros de indexado, garantizando condiciones comparables en cuanto a tamaño de corpus, batch size y dimensionalidad.

---

### 5. Resultados experimentales

Los resultados agregados se resumen a continuación:

| Modelo | Recall@10 | MRR@10 | nDCG@10 | Dim | Dispositivo | Tiempo de *embedding* (s) |
|--------|------------|--------|----------|------|--------------|---------------------------|
| BAAI/bge-m3 | 0.79 | 0.59 | 0.60 | 1024 | GPU | 775.9 |
| intfloat/multilingual-e5-large-instruct | 0.81 | 0.61 | 0.62 | 1024 | GPU | 766.1 |
| sentence-transformers/all-MiniLM-L6-v2 | 0.82 | 0.60 | 0.61 | 384 | GPU | 36.0 |
| intfloat/e5-base-v2 | 0.82 | 0.60 | 0.61 | 768 | GPU | 177.4 |
| mixedbread-ai/mxbai-embed-large-v1 | **0.85** | **0.63** | **0.65** | 1024 | GPU | 581.6 |

El modelo `mixedbread-ai/mxbai-embed-large-v1` alcanzó la mayor puntuación en las tres métricas principales, con un **nDCG@10 de 0.645**, lo que indica que sus vectores preservan mejor la estructura semántica y jerárquica de los textos técnicos.  
El modelo `multilingual-e5-large-instruct` se posicionó en segundo lugar, destacando como la mejor alternativa multilingüe, mientras que `all-MiniLM-L6-v2` ofreció una relación excelente entre velocidad y precisión (con tiempos de inferencia 20× menores).

---

### 6. Justificación y análisis de las métricas

Las tres métricas empleadas (Recall@10, MRR@10 y nDCG@10) son complementarias y permiten una evaluación robusta del rendimiento de los modelos:

- **Recall@10** evalúa la **capacidad de cobertura**: mide si el modelo logra recuperar los fragmentos relevantes, independientemente del orden.  
- **MRR@10** captura la **precisión temprana**: un modelo con alto MRR prioriza correctamente los documentos más relevantes.  
- **nDCG@10**, al ser una métrica logarítmica, equilibra ambas dimensiones ponderando la relevancia por posición.  
  Su robustez frente a ruido y empates la convierte en la métrica más representativa del rendimiento global en tareas de recuperación semántica.

La coherencia de los valores entre las tres métricas, junto con la consistencia empírica observada en la literatura (benchmarks MTEB, BEIR), permite inferir con rigor que el modelo `mixedbread-ai/mxbai-embed-large-v1` ofrece la representación vectorial más fiel a la estructura semántica de los *recalls*.

---

### 7. Conclusiones

1. El **proceso de chunking** fue determinante: al segmentar los textos, las métricas aumentaron significativamente (≈+25% en nDCG@10), indicando que los modelos capturan mejor las relaciones semánticas locales que los contextos excesivamente largos.  
2. El modelo **`mixedbread-ai/mxbai-embed-large-v1`** se consolida como la mejor opción para representar de forma unificada *recalls*, *complaints* y *manuales técnicos* en un mismo espacio vectorial.  
3. Para escenarios multilingües o consultas en español, **`intfloat/multilingual-e5-large-instruct`** es la alternativa más adecuada sin necesidad de traducción.  
4. En entornos con restricciones de cómputo, **`all-MiniLM-L6-v2`** ofrece un compromiso eficiente entre velocidad, costo y precisión.  

Estos resultados fundamentan la elección del modelo base para la **fase de integración semántica** con *LangChain*, donde las representaciones se emplearán para tareas de recuperación aumentada (RAG) y generación de respuestas explicativas que combinen información técnica y contextual.

---

### 8. Productos generados

- `recalls_chunks_fixed.parquet`: corpus segmentado.  
- `metrics.csv`: resultados crudos por modelo.  
- `ranking_quality.csv` y `ranking_efficiency.csv`: comparativos consolidados.  
- `ndcg_bar.png`: gráfico de rendimiento.  
- `ranking_report.md`: resumen tabular final.

---


In [ ]:
!pip -q install "sentence-transformers>=3.0.1" "huggingface_hub>=0.23.0" faiss-cpu pandas numpy scikit-learn tqdm
# !pip -q install jina-embeddings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/Proyecto final")
F_BYCAMP  = BASE/"recalls_by_campaign.csv"            # puede NO traer narrativas si lo exportaste reducido
F_RAW_ENR = BASE/"recalls_raw_enriched.csv"           # este SÍ suele traer DESC_/CONSEQUENCE_/CORRECTIVE_/NOTES
F_META    = BASE/"neo4j_exports/recalls_cleaned_good.csv"  # llaves normalizadas (make/model/year/component)

def cols(path):
    if not path.exists(): return []
    try:
        return pd.read_csv(path, nrows=1).columns.tolist()
    except Exception as e:
        print("Error leyendo", path.name, "->", e)
        return []

print("by_campaign.csv cols:", cols(F_BYCAMP))
print("raw_enriched.csv cols:", cols(F_RAW_ENR))
print("cleaned_good.csv cols:", cols(F_META))


by_campaign.csv cols: ['CAMPNO', 'POTAFF_num', 'MFGNAME', 'MAKETXT', 'MODELTXT', 'COMPNAME', 'COMP_L1', 'COMP_L2', 'COMP_L3', 'RCLTYPECD', 'FMVSS', 'RCDATE', 'ODATE', 'DATEA', 'BGMAN', 'ENDMAN', 'YEARTXT']
raw_enriched.csv cols: ['RECORD_ID', 'CAMPNO', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'MFGCAMPNO', 'COMPNAME', 'MFGNAME', 'BGMAN', 'ENDMAN', 'RCLTYPECD', 'POTAFF', 'ODATE', 'INFLUENCED_BY', 'MFGTXT', 'RCDATE', 'DATEA', 'RPNO', 'FMVSS', 'DESC_DEFECT', 'CONSEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES', 'RCL_CMPT_ID', 'EXTRA_25', 'EXTRA_26', 'EXTRA_27', 'EXTRA_28', 'EXTRA_29', 'POTAFF_num', 'len_DESC_DEFECT', 'len_CONSEQUENCE_DEFECT', 'len_CORRECTIVE_ACTION', 'len_NOTES', 'COMP_L1', 'COMP_L2', 'COMP_L3']
cleaned_good.csv cols: ['campaign_no', 'make', 'model', 'year', 'component', 'recall_date', 'make_norm', 'model_norm']


1) Reconstruir text_full desde el archivo que SÍ tiene narrativas

In [ ]:
import pandas as pd, re

# Elige la fuente con narrativas disponibles
SRC_TEXT = F_RAW_ENR if F_RAW_ENR.exists() else F_BYCAMP   # prioriza raw_enriched (suele traer texto)
df_text  = pd.read_csv(SRC_TEXT, dtype=str, keep_default_na=False, low_memory=False)

# Normaliza nombres -> estándar
rename = {
    "CAMPNO":"campaign_no",
    "DESC_DEFECT":"subject",
    "CONSEQUENCE_DEFECT":"consequence",
    "CORRECTIVE_ACTION":"corrective_action",
    "NOTES":"notes"
}
for k,v in rename.items():
    if k in df_text.columns: df_text = df_text.rename(columns={k:v})

# Asegura columnas textuales presentes
text_cols = [c for c in ["subject","consequence","corrective_action","notes"] if c in df_text.columns]
assert text_cols, f"No encuentro columnas narrativas en {SRC_TEXT.name}. Revisa la Celda 0."

# Construye text_full
def join_text(row):
    parts = [str(row.get(c,"") or "").strip() for c in text_cols]
    parts = [p for p in parts if p]
    return re.sub(r"\s+"," ",". ".join(parts)).strip()

df_text["campaign_no"] = df_text["campaign_no"].astype(str).str.strip()
df_text["text_full"]   = df_text.apply(join_text, axis=1)
df_text = df_text[["campaign_no","text_full"]].drop_duplicates("campaign_no")

print("Campañas con texto_full:", (df_text["text_full"].str.len()>0).sum(), "/", len(df_text))
df_text.head(2)


Campañas con texto_full: 14290 / 14290


,campaign_no,text_full
0,10C001000,DOREL JUVENILE GROUP (DJG) IS RECALLING CERTAI...
1,10C003000,CERTAIN CYBEX SOLUTION X-FIX MODEL BOOSTER SEA...


2) Enriquecer con llaves (make/model/year/component) del meta y filtrar vacíos

In [ ]:
meta = pd.read_csv(F_META, dtype=str, keep_default_na=False, low_memory=False)
meta["campaign_no"] = meta["campaign_no"].astype(str).str.strip()

recalls = meta.merge(df_text, on="campaign_no", how="left")
recalls["text_full"] = (recalls["text_full"].fillna("")
                        .str.replace(r"\s+"," ", regex=True).str.strip())

# Filtra los que realmente tienen narrativa
recalls = recalls[recalls["text_full"].str.len()>0].copy()

print("Recalls con texto:", recalls.shape)
recalls.head(2)[["campaign_no","make","model","year","component","text_full"]]


Recalls con texto: (12729, 9)


,campaign_no,make,model,year,component,text_full
0,10E002000,TOYOTA,CELICA,1986,SUSPENSION,RIDE CONTROL IS RECALLING CERTAIN FRONT STRUT ...
1,10E013000,DODGE,RAM 2500,2007,STEERING:GEAR BOX:SHAFT PITMAN,FABTECH IS RECALLING CERTAIN AFTERMARKET REPLA...


In [ ]:
import re, pandas as pd
from pathlib import Path

CHUNK_WORDS = 250
CHUNK_OVERL = 40

def chunk_words(text, n_words=CHUNK_WORDS, overlap=CHUNK_OVERL):
    words = re.findall(r"\S+", str(text))
    if not words: return []
    out=[]; step = max(1, n_words-overlap)
    for i in range(0, len(words), step):
        piece = " ".join(words[i:i+n_words]).strip()
        if piece: out.append(piece)
        if i+n_words >= len(words): break
    return out

rows=[]
for _, r in recalls.iterrows():
    for j, ch in enumerate(chunk_words(r["text_full"])):
        rows.append({
            "chunk_id": f"{r['campaign_no']}::ch{j}",
            "campaign_no": r["campaign_no"],
            "make": (r.get("make","") or "").upper().strip(),
            "model": (r.get("model","") or "").upper().strip(),
            "year": r.get("year",""),
            "component": (r.get("component","") or "").upper().strip(),
            "chunk_idx": j,
            "text_chunk": ch
        })
chunks = pd.DataFrame(rows)
print("Chunks:", chunks.shape)

OUT = Path("/content/drive/MyDrive/NHTSA/processed")
recalls.to_parquet(OUT/"recalls_master_fixed.parquet", index=False)
chunks.to_parquet(OUT/"recalls_chunks_fixed.parquet", index=False)
print("Guardados:", OUT/"recalls_master_fixed.parquet", "|", OUT/"recalls_chunks_fixed.parquet")


Chunks: (12871, 8)
Guardados: /content/drive/MyDrive/NHTSA/processed/recalls_master_fixed.parquet | /content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet


Pasando al encaje

In [ ]:
import pandas as pd

chunks = pd.read_parquet("/content/drive/MyDrive/NHTSA/processed/recalls_chunks_fixed.parquet")
texts = chunks["text_chunk"].tolist()
ids   = chunks["chunk_id"].tolist()

print(len(texts), "chunks listos para encajar.")
print(texts[0][:300])  # vista previa del primer chunk


12871 chunks listos para encajar.
RIDE CONTROL IS RECALLING CERTAIN FRONT STRUT MOUNTS BRANDED AS GABRIEL RIDE CONTROL OR ARVINMERITOR, P/NOS. 142435, 142193, 142305, 142303, SOLD AS REPLACEMENT EQUIPMENT FOR THE VEHICLES LISTED ABOVE. THE AFFECTED FRONT STRUT MOUNTS DID NOT CONTAIN A WELD JOINT BETWEEN THE BEARING HOUSING AND THE R


0) Setup (montar Drive e instalar libs)



In [ ]:
# Colab: GPU T4 para embeddings; FAISS en CPU para máxima compatibilidad
!nvidia-smi -L || true
!pip -q install faiss-cpu torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install sentence-transformers transformers tqdm pandas pyarrow


GPU 0: Tesla T4 (UUID: GPU-069e91f3-3229-bdc7-a386-08b76b4d3828)
ERROR: Could not find a version that satisfies the requirement faiss-cpu (from versions: none)
ERROR: No matching distribution found for faiss-cpu


1) Cargar tus artefactos y armar corpus de evaluación (campaña)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path("/content/drive/MyDrive/NHTSA/processed")
MASTER = BASE/"recalls_master_fixed.parquet"   # 1 fila por campaña, con text_full
CHUNKS = BASE/"recalls_chunks_fixed.parquet"   # (no lo usamos para R2R, pero queda por si quieres RAG)

assert MASTER.exists(), f"No encuentro {MASTER}"
recalls = pd.read_parquet(MASTER)

# Normaliza llaves
for c in ["campaign_no","make","model","component","text_full"]:
    if c in recalls.columns:
        recalls[c] = recalls[c].astype(str).str.strip()
if "year" in recalls.columns:
    recalls["year"] = pd.to_numeric(recalls["year"], errors="coerce").astype("Int64")

# Filtra campañas con texto
recalls = recalls[recalls["text_full"].str.len() > 0].copy()
print("Campañas con texto:", recalls.shape)

# Chequeo de variedad para formar grupos de relevancia
print(recalls[["make","model","year","component"]].head())

# Para evaluación R2R trabajamos a nivel campaña:
texts = recalls["text_full"].tolist()
ids   = recalls["campaign_no"].tolist()

len(texts), len(ids)


Campañas con texto: (12729, 9)
     make     model  year                               component
0  TOYOTA    CELICA  1986                              SUSPENSION
1   DODGE  RAM 2500  2007          STEERING:GEAR BOX:SHAFT PITMAN
2  NISSAN     TITAN  2010  SUSPENSION:FRONT:CONTROL ARM:LOWER ARM
3     BMW      740I  1996  STEERING:LINKAGES:DRAG:LINK:CONNECTION
4   DODGE   CARAVAN  2007                               EQUIPMENT


(12729, 12729)

2) Construir Qrels (relevancias) por llaves con “fallback”

In [ ]:
from collections import defaultdict

def build_qrels(df):
    """
    Intenta (make, model, year, component) y cae a claves más débiles si hay pocos matches.
    Devuelve dict: { campaign_no: set([relevantes_campaign_no, ...]) }
    """
    keys_list = [
        ["make", "model", "year", "component"],
        ["make", "model", "component"],
        ["make", "model"],
        ["make", "component"]
    ]
    # Mapeos por clave
    for keys in keys_list:
        if not set(keys).issubset(df.columns):
            continue
        grp = df.dropna(subset=keys).groupby(keys)["campaign_no"].apply(list)
        rel = {}
        for _, lst in grp.items():
            s = set(lst)
            for cid in lst:
                rel.setdefault(cid, set())
                rel[cid].update(s - {cid})
        # ¿Hay suficientes con al menos 1 relevante?
        ok = sum(1 for v in rel.values() if len(v)>0)
        coverage = ok / max(1, len(df))
        if coverage > 0.2:  # regla simple: al menos 20% de campañas con pares
            return rel, keys
    return defaultdict(set), []

rel_by_camp, used_keys = build_qrels(recalls)
print("Clave usada para qrels:", used_keys)
print("Campañas con >=1 relevante:", sum(len(v)>0 for v in rel_by_camp.values()))


Clave usada para qrels: ['make', 'model', 'component']
Campañas con >=1 relevante: 2839


3) Utilidades: métrica (Recall@10 / MRR@10 / nDCG@10) y FAISS

In [ ]:
!pip install faiss-cpu --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 77.9 MB/s eta 0:00:00


In [ ]:
import time
import faiss
import numpy as np

def l2_normalize(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    return x / n

def build_faiss_ip(emb: np.ndarray):
    d = emb.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(emb.astype(np.float32))
    return index

def eval_search(index, q_emb, k, ids, rels):
    t0 = time.time()
    D, I = index.search(q_emb.astype(np.float32), k)
    total_ms = (time.time() - t0) * 1000.0
    per_query_ms = total_ms / max(1, len(q_emb))

    def mrr_at_k(relset, retrieved):
        for rank, idx in enumerate(retrieved, 1):
            if ids[idx] in relset:
                return 1.0 / rank
        return 0.0

    def dcg_at_k(relset, retrieved):
        dcg = 0.0
        for rank, idx in enumerate(retrieved, 1):
            if ids[idx] in relset:
                dcg += 1.0 / np.log2(rank + 1)
        return dcg

    def idcg_at_k(m, k):
        m = min(m, k)
        return sum(1.0 / np.log2(i+1) for i in range(1, m+1))

    recalls, mrrs, ndcgs = [], [], []
    latencies = [per_query_ms] * len(I)
    for qi in range(len(I)):
        qid = ids[qi]
        relset = rels.get(qid, set())
        if not relset:
            continue
        retrieved = [idx for idx in I[qi] if idx != qi]
        hit = sum(1 for idx in retrieved if ids[idx] in relset)
        recalls.append(1.0 if hit > 0 else 0.0)
        mrrs.append(mrr_at_k(relset, retrieved))
        ndcgs.append(dcg_at_k(relset, retrieved)/idcg_at_k(len(relset), len(retrieved)) if len(relset)>0 else 0.0)

    def pctl(a, p):
        if not a: return 0.0
        return float(np.percentile(a, p))

    return {
        "Recall@10": float(np.mean(recalls)) if recalls else 0.0,
        "MRR@10":    float(np.mean(mrrs))    if mrrs else 0.0,
        "nDCG@10":   float(np.mean(ndcgs))   if ndcgs else 0.0,
        "lat_p50_ms": pctl(latencies, 50),
        "lat_p95_ms": pctl(latencies, 95),
        "n_queries":  len(recalls),
    }


4) Wrapper de evaluación por modelo (GPU T4 si disponible)

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
from pathlib import Path

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUT_DIR = Path("/content/drive/MyDrive/Proyecto final/eval_results")
OUT_DIR.mkdir(parents=True, exist_ok=True)
metrics_path = OUT_DIR/"metrics_summary.csv"

def run_model_eval(model_name, texts, ids, rel_by_camp, sample_queries=1500, seed=42, batch_size=64):
    rng = np.random.default_rng(seed)
    n = len(texts)
    q_idx = rng.choice(n, size=min(sample_queries, n), replace=False)
    order = np.r_[q_idx, np.setdiff1d(np.arange(n), q_idx, assume_unique=True)]
    texts_ord = [texts[i] for i in order]
    ids_ord   = [ids[i]   for i in order]

    t0 = time.time()
    model = SentenceTransformer(model_name, device=DEVICE)
    load_time = time.time() - t0

    t0 = time.time()
    embs = model.encode(
        texts_ord,
        batch_size=(64 if DEVICE=="cuda" else 16),
        convert_to_numpy=True,
        show_progress_bar=True,
        normalize_embeddings=True
    )
    embed_time = time.time() - t0
    dim = int(embs.shape[1])

    t0 = time.time()
    index = build_faiss_ip(embs.astype(np.float32))
    index_time = time.time() - t0

    q_emb = embs[:len(q_idx)]
    metrics = eval_search(index, q_emb, k=10, ids=ids_ord, rels=rel_by_camp)

    n_docs = int(embs.shape[0])
    est_store_mb = n_docs * dim * 4 / (1024**2)

    out = {
        "model": model_name,
        "task": "R2R",
        "Recall@10": metrics["Recall@10"],
        "MRR@10": metrics["MRR@10"],
        "nDCG@10": metrics["nDCG@10"],
        "embed_time_s": round(embed_time, 3),
        "index_build_s": round(index_time, 3),
        "load_time_s": round(load_time, 3),
        "lat_p50_ms": round(metrics["lat_p50_ms"], 3),
        "lat_p95_ms": round(metrics["lat_p95_ms"], 3),
        "n_queries": int(metrics["n_queries"]),
        "n_docs": n_docs,
        "dim": dim,
        "device": DEVICE,
        "est_store_mb": round(est_store_mb, 2),
    }
    return out

def already_evaluated(model_name, path: Path) -> bool:
    if not path.exists() or path.stat().st_size == 0:
        return False
    try:
        df = pd.read_csv(path)
        return (df["model"] == model_name).any()
    except Exception:
        return False


5) Ejecutar la comparativa

In [ ]:
MODELS = [
    "BAAI/bge-m3",
    "intfloat/multilingual-e5-large-instruct",
    "intfloat/e5-base-v2",
    "sentence-transformers/all-MiniLM-L6-v2",
    "mixedbread-ai/mxbai-embed-large-v1",
    "nomic-ai/ModernBERT-base",
]

SKIP_IF_PRESENT = True   # ponlo en False si quieres recalcular aunque exista

if metrics_path.exists() and metrics_path.stat().st_size > 0:
    metrics_rows = pd.read_csv(metrics_path).to_dict("records")
else:
    metrics_rows = []

for m in MODELS:
    if SKIP_IF_PRESENT and already_evaluated(m, metrics_path):
        print(f"↪︎ Saltando {m} (ya existe en metrics.csv)")
        continue
    print(f"\n=== Evaluando: {m} ===")
    try:
        res = run_model_eval(
            model_name=m,
            texts=texts,
            ids=ids,
            rel_by_camp=rel_by_camp,
            sample_queries=1500,
            batch_size=64 if DEVICE=="cuda" else 16
        )
        metrics_rows.append(res)
        pd.DataFrame(metrics_rows).to_csv(metrics_path, index=False)
        print("→ OK:", res)
    except Exception as e:
        print("→ ERROR en", m, ":", repr(e))

print("\n👉 Resultados guardados en:", metrics_path)
try:
    display(pd.read_csv(metrics_path))
except Exception:
    pass


↪︎ Saltando BAAI/bge-m3 (ya existe en metrics.csv)
↪︎ Saltando intfloat/multilingual-e5-large-instruct (ya existe en metrics.csv)

=== Evaluando: intfloat/e5-base-v2 ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Batches:   0%|          | 0/199 [00:00<?, ?it/s]

→ OK: {'model': 'mixedbread-ai/mxbai-embed-large-v1', 'task': 'R2R', 'Recall@10': 0.8539944903581267, 'MRR@10': 0.630976430976431, 'nDCG@10': 0.6451492069114759, 'embed_time_s': 581.566, 'index_build_s': 0.176, 'load_time_s': 13.651, 'lat_p50_ms': 0.602, 'lat_p95_ms': 0.602, 'n_queries': 363, 'n_docs': 12729, 'dim': 1024, 'device': 'cuda', 'est_store_mb': 49.72}

=== Evaluando: nomic-ai/ModernBERT-base ===
→ ERROR en nomic-ai/ModernBERT-base : OSError("nomic-ai/ModernBERT-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'\nIf this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`")

👉 Resultados guardados en: /content/drive/MyDrive/Proyecto final/eval_results/metrics_summary.csv


,model,task,Recall@10,MRR@10,nDCG@10,embed_time_s,index_build_s,load_time_s,n_queries,n_docs,dim,device,lat_p50_ms,lat_p95_ms,est_store_mb
0,BAAI/bge-m3,R2R,0.458333,0.221991,0.191548,1883.288884,5.712859,7.992231,1500,5000,1024,cpu,NaN,NaN,NaN
1,BAAI/bge-m3,R2R,0.793388,0.592278,0.597449,775.929000,0.078000,6.070000,363,12729,1024,cuda,0.520,0.520,49.72
2,intfloat/multilingual-e5-large-instruct,R2R,0.812672,0.607802,0.615946,766.142000,0.103000,8.721000,363,12729,1024,cuda,0.279,0.279,49.72
3,sentence-transformers/all-MiniLM-L6-v2,R2R,0.818182,0.602524,0.606570,36.029000,0.015000,6.857000,363,12729,384,cuda,0.135,0.135,18.65
4,intfloat/e5-base-v2,R2R,0.823691,0.596612,0.606554,177.412000,0.053000,12.321000,363,12729,768,cuda,0.255,0.255,37.29
5,mixedbread-ai/mxbai-embed-large-v1,R2R,0.853994,0.630976,0.645149,581.566000,0.176000,13.651000,363,12729,1024,cuda,0.602,0.602,49.72


In [ ]:
# === Ranking consolidado (calidad vs. costo/tiempo/tamaño) ===
# - Lee metrics.csv
# - Calcula compuestos, Pareto
# - Exporta tablas y gráfico
# --------------------------------------------------------------

import os, sys, math, json, textwrap
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# ---------- 1) Paths & lectura robusta ----------
CANDIDATE_PATHS = [
    "/content/drive/MyDrive/Proyecto final/eval_results/metrics_summary.csv",  # tu ruta típica
    "/content/metrics.csv",                                            # por si lo dejaste en /content
    "/mnt/data/metrics.csv",                                           # fallback local
]

def first_existing(paths):
    for p in paths:
        if Path(p).exists() and Path(p).stat().st_size > 0:
            return Path(p)
    return None

METRICS = first_existing(CANDIDATE_PATHS)
if METRICS is None:
    raise FileNotFoundError(
        "No encontré metrics.csv. Asegúrate de haber corrido la evaluación y que el archivo exista en:\n" +
        "\n".join(CANDIDATE_PATHS)
    )

OUT_DIR = Path(METRICS).parent
print(f"Usando métricas en: {METRICS}")
print(f"Exportando resultados a: {OUT_DIR}")

df = pd.read_csv(METRICS)

# Columns esperadas (rellena faltantes)
for c in ["lat_p50_ms","lat_p95_ms","est_store_mb","n_docs","dim","embed_time_s","nDCG@10","MRR@10","Recall@10"]:
    if c not in df.columns:
        df[c] = np.nan

# Si no hay tamaño estimado, lo aproximamos: n_docs * dim * 4 bytes
def approx_store_mb(row):
    if pd.notna(row.get("est_store_mb", np.nan)):
        return row["est_store_mb"]
    n_docs = row.get("n_docs", np.nan)
    dim    = row.get("dim", np.nan)
    if pd.notna(n_docs) and pd.notna(dim):
        return float(n_docs) * float(dim) * 4 / (1024**2)
    return np.nan

df["est_store_mb"] = df.apply(approx_store_mb, axis=1)

# ---------- 2) Elegir la mejor corrida por modelo ----------
# criterio: mayor nDCG@10 (desempate: MRR, luego Recall)
df_best = (
    df.sort_values(["model","nDCG@10","MRR@10","Recall@10"], ascending=[True, False, False, False])
      .groupby("model", as_index=False).head(1)
      .reset_index(drop=True)
)

# ---------- 3) Compuestos ----------
def norm01(s):
    s = pd.to_numeric(s, errors="coerce")
    if s.isna().all():  # todo NaN
        return pd.Series([0.5]*len(s), index=s.index)
    s = s.fillna(s.median())
    mn, mx = s.min(), s.max()
    if mx == mn:
        return pd.Series([0.5]*len(s), index=s.index)
    return (s - mn) / (mx - mn)

# Calidad
q_ndcg = norm01(df_best["nDCG@10"])
q_mrr  = norm01(df_best["MRR@10"])
q_rec  = norm01(df_best["Recall@10"])
df_best["Q_composite"] = 0.7*q_ndcg + 0.2*q_mrr + 0.1*q_rec

# Eficiencia (menor tiempo / menor tamaño / menor latencia → mejor)
speed_inv   = norm01(1.0 / pd.to_numeric(df_best["embed_time_s"], errors="coerce").replace(0, np.nan))
size_inv    = norm01(1.0 / pd.to_numeric(df_best["est_store_mb"], errors="coerce").replace(0, np.nan))
latency_inv = norm01(1.0 / pd.to_numeric(df_best["lat_p50_ms"], errors="coerce").fillna(
    pd.to_numeric(df_best["lat_p50_ms"], errors="coerce").median()
).replace(0, np.nan))
df_best["E_composite"] = 0.5*speed_inv + 0.3*size_inv + 0.2*latency_inv

# ---------- 4) Pareto (max nDCG, min tiempo & tamaño) ----------
vals = np.stack([
    pd.to_numeric(df_best["nDCG@10"], errors="coerce").fillna(0).to_numpy(),                      # maximize
    (1.0 / pd.to_numeric(df_best["embed_time_s"], errors="coerce").replace(0, np.nan)).fillna(0), # maximize inverse time
    (1.0 / pd.to_numeric(df_best["est_store_mb"], errors="coerce").replace(0, np.nan)).fillna(0), # maximize inverse size
], axis=1)

pareto = np.ones(len(df_best), dtype=bool)
for i in range(len(df_best)):
    for j in range(len(df_best)):
        if i == j:
            continue
        # j domina a i si es >= en todo y > en al menos una dimensión
        if np.all(vals[j] >= vals[i]) and np.any(vals[j] > vals[i]):
            pareto[i] = False
            break
df_best["Pareto"] = np.where(pareto, "✔︎", "")

# ---------- 5) Rankings ----------
show_cols = ["model","task","n_docs","dim","device",
             "nDCG@10","MRR@10","Recall@10",
             "embed_time_s","lat_p50_ms","est_store_mb",
             "Q_composite","E_composite","Pareto"]

rank_quality    = df_best.sort_values(["Q_composite","nDCG@10","MRR@10","Recall@10"], ascending=False)[show_cols].reset_index(drop=True)
rank_efficiency = df_best.sort_values(["E_composite","embed_time_s","est_store_mb"], ascending=[False, True, True])[show_cols].reset_index(drop=True)

# ---------- 6) Exportar ----------
rank_quality_path    = OUT_DIR/"ranking_quality.csv"
rank_efficiency_path = OUT_DIR/"ranking_efficiency.csv"
df_best_path         = OUT_DIR/"best_by_model.csv"

rank_quality.to_csv(rank_quality_path, index=False)
rank_efficiency.to_csv(rank_efficiency_path, index=False)
df_best.to_csv(df_best_path, index=False)

# Gráfico simple de nDCG@10 (una barra por modelo)
plt.figure(figsize=(10, 4))
x = np.arange(len(df_best))
plt.bar(x, pd.to_numeric(df_best["nDCG@10"], errors="coerce"))
plt.xticks(x, df_best["model"], rotation=45, ha="right")
plt.ylabel("nDCG@10")
plt.title("nDCG@10 (mejor corrida por modelo)")
plt.tight_layout()
chart_path = OUT_DIR/"ndcg_bar.png"
plt.savefig(chart_path, dpi=160)
plt.close()

print("\n✅ Listo. Archivos generados:")
print(" - Ranking (Calidad):    ", rank_quality_path)
print(" - Ranking (Eficiencia): ", rank_efficiency_path)
print(" - Mejor corrida/modelo: ", df_best_path)
print(" - Gráfico nDCG:         ", chart_path)

# Mostrar resumen compacto en pantalla
from IPython.display import display
print("\nTOP por Calidad (Q_composite):")
display(rank_quality.head(10))

print("\nTOP por Eficiencia (E_composite):")
display(rank_efficiency.head(10))


Usando métricas en: /content/drive/MyDrive/Proyecto final/eval_results/metrics_summary.csv
Exportando resultados a: /content/drive/MyDrive/Proyecto final/eval_results

✅ Listo. Archivos generados:
 - Ranking (Calidad):     /content/drive/MyDrive/Proyecto final/eval_results/ranking_quality.csv
 - Ranking (Eficiencia):  /content/drive/MyDrive/Proyecto final/eval_results/ranking_efficiency.csv
 - Mejor corrida/modelo:  /content/drive/MyDrive/Proyecto final/eval_results/best_by_model.csv
 - Gráfico nDCG:          /content/drive/MyDrive/Proyecto final/eval_results/ndcg_bar.png

TOP por Calidad (Q_composite):


,model,task,n_docs,dim,device,nDCG@10,MRR@10,Recall@10,embed_time_s,lat_p50_ms,est_store_mb,Q_composite,E_composite,Pareto
0,mixedbread-ai/mxbai-embed-large-v1,R2R,12729,1024,cuda,0.645149,0.630976,0.853994,581.566,0.602,49.72,1.000000,0.008137,✔︎
1,intfloat/multilingual-e5-large-instruct,R2R,12729,1024,cuda,0.615946,0.607802,0.812672,766.142,0.279,49.72,0.383496,0.067245,
2,sentence-transformers/all-MiniLM-L6-v2,R2R,12729,384,cuda,0.606570,0.602524,0.818182,36.029,0.135,18.65,0.227708,1.000000,✔︎
3,intfloat/e5-base-v2,R2R,12729,768,cuda,0.606554,0.596612,0.823691,177.412,0.255,37.29,0.206019,0.220838,
4,BAAI/bge-m3,R2R,12729,1024,cuda,0.597449,0.592278,0.793388,775.929,0.520,49.72,0.000000,0.009117,



TOP por Eficiencia (E_composite):


,model,task,n_docs,dim,device,nDCG@10,MRR@10,Recall@10,embed_time_s,lat_p50_ms,est_store_mb,Q_composite,E_composite,Pareto
0,sentence-transformers/all-MiniLM-L6-v2,R2R,12729,384,cuda,0.606570,0.602524,0.818182,36.029,0.135,18.65,0.227708,1.000000,✔︎
1,intfloat/e5-base-v2,R2R,12729,768,cuda,0.606554,0.596612,0.823691,177.412,0.255,37.29,0.206019,0.220838,
2,intfloat/multilingual-e5-large-instruct,R2R,12729,1024,cuda,0.615946,0.607802,0.812672,766.142,0.279,49.72,0.383496,0.067245,
3,BAAI/bge-m3,R2R,12729,1024,cuda,0.597449,0.592278,0.793388,775.929,0.520,49.72,0.000000,0.009117,
4,mixedbread-ai/mxbai-embed-large-v1,R2R,12729,1024,cuda,0.645149,0.630976,0.853994,581.566,0.602,49.72,1.000000,0.008137,✔︎
